# Debug the real spectra

So it doesn't work well with real data, so having taken out one example of real data, the sequence is

__NOTE__: These are ALL hybrids


1. `DLQTLALLL`
2. `DPQVAQLELGGEVEDPQVAQLELGGGPGAG`
3. `DLPVNSPMTKG`
4. `DLQTLALWSRM`
5. `DLQTLALNAAR`
6. `DLQTLALNAAR`
7. `DLQTLEVE`
8. `DLQTLALEVE`

so toy with the tool until we find out whats wrong

In [1]:
correct = ['DLQTLALLL', 'DPQVAQLELGGEVEDPQVAQLELGGGPGAG', 'DLPVNSPMTKG', 'DLQTLALWSRM', 'DLQTLALNAAR', 'DLQTLALNAAR', 'DLQTLEVE', 'DLQTLALEVE']


## Load params and run

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import runner
    

In [3]:
specPath = '/Users/zacharymcgrath/Desktop/nod2 data/hybridSpec/'
fastaPath = '/Users/zacharymcgrath/Desktop/nod2 data/filteredNOD2.fasta'
outputDir = '/Users/zacharymcgrath/Desktop/Experiment output/hybridReal/'
minPep = 3
maxPep = 30
tolerance = 20
relative_abundance_filter = 0.005
precursor_tolerance = 1
peak_filter = 0
verbose = True
scoringAlg = 'ion'

In [4]:
params = {
    'spectra_folder': specPath,
    'database_file': fastaPath,
    'output_dir': outputDir,
    'min_peptide_len': minPep,
    'max_peptide_len': maxPep,
    'tolerance': tolerance,
    'precursor_tolerance': precursor_tolerance,
    'peak_filter': peak_filter, 
    'relative_abundance_filter': relative_abundance_filter,
    'verbose': verbose, 
    'scoring_alg': scoringAlg, 
    'DEBUG': True
}

In [ ]:
%%time
runner.run(params)

## Load and read results

In [ ]:
import pandas as pd

outputFile = outputDir + 'summary.tsv'
hybridOutputFile = outputDir + 'hybrid_summary.tsv'

In [ ]:
df = pd.read_csv(outputFile, sep='\t')
hdf = pd.read_csv(hybridOutputFile, sep='\t')

In [ ]:
hdf.head(10)

In [ ]:
df.head(10)

In [ ]:
import json

s = json.load(open(outputDir + 'summary.json', 'r'))

In [ ]:
for i, (k, v) in enumerate(s.items()):
    c = correct[i]
    print(f'Alignment for spectrum {i} \t "correct": {c} \n ==============================')
    for a in v['alignments']:
        print(f'{a["sequence"]}: \t {a["total_score"]}, \t {a["b_score"]}, \t {a["y_score"]}')
    print()

    

In [ ]:
from src.sequence.gen_spectra import gen_spectrum

In [ ]:
seq1 = 'QVEQLELGGSPG'
seq2 = 'ATSPQLELGGSPG'

gen_spectrum(seq2, ion='b', charge=1)['spectrum']